<a href="https://colab.research.google.com/github/kmk4444/LLM/blob/main/generate_sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- **Requirements.txt**

In [1]:
!touch requirements.txt
!echo python-dotenv >> requirements.txt
!echo openai >> requirements.txt
!echo streamlit >> requirements.txt
!echo assemblyai >> requirements.txt
!echo replicate >> requirements.txt

- **terminal / bash komutu**

In [ ]:
pip install -r requirements.txt

In [7]:
%%writefile audio_ops.py
from openai import OpenAI
import assemblyai as aai
import streamlit as st
import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_openai = os.getenv("openai_apikey")
#my_key_assemblyai= os.getenv("assemblyai_apikey")

client = OpenAI(
    api_key="---"
)


def create_speech_from_text(prompt, speech_file_name, voice_type="alloy"):
  AI_Response = client.audio.speech.create(
      model="tts-1",
      voice=voice_type,
      response_format="mp3", #type of result: aac, flac, mp3 or opus
      input=prompt
  )
  #  analysis process of result

  AI_Response.stream_to_file(speech_file_name) # we save output to speech_file_name
  # Actually, it creates and saves a sound file in local system. local adress is speech_file_name
  return "Seslendirme işlemi tamamlandı."

# We crate four tabs for our interface.
tab_TTS, tab_whisper, tab_translation, tab_conformer = st.tabs(
    [
     "TTS ile Ses Sentezleme",
     "Whisper ile Transkripsiyon",
     "Whisper ile Tercüme",
     "Conformer ile Transkripsiyon"
     ]
)

# This tab is related to TTS-1 model
with tab_TTS:
  st.subheader("TTS-1 Modeli ile Konuşma Sentezleme")
  st.divider()

  prompt = st.text_input("Seslendirmek istediğiniz metni giriniz",key="prompt_tts")
  voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"] # voice types
  voice_type = st.selectbox(label="Ses tercihiniz:", options=voices, key="voice_tts") # user selects voice types.
  generate_btn = st.button("Ses Sentezle", key="button_tts")

  if generate_btn:
    status = create_speech_from_text(prompt=prompt,speech_file_name="speech.mp3",voice_type=voice_type)
    st.success(status)

    audio_file = open("speech.mp3","rb") # rb: read binary. Emter file adress and make read binary. We want to temporarily save a sound file
    audio_bytes = audio_file.read() # we save sound bytes for streamlit.

    st.audio(data=audio_bytes,format="audio/mp3")
    st.balloons()


Overwriting audio_ops.py


In [8]:
!npm install localtunnel
!streamlit run /content/audio_ops.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.523s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 1.957s
your url is: https://loose-spoons-punch.loca.lt
^C
